<a href="https://colab.research.google.com/github/liliya111darky/logical-ai/blob/notebooks/05_veritas_ai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ============================================
# 5. Telegram-бот для Numen
# ============================================

import sympy as sp
from sympy import sympify
from z3 import *
import sqlite3
import json
import os
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, ContextTypes, filters

# ===== Настройки =====
TELEGRAM_TOKEN = "7886763904:AAGsvCNzpSY1qxXduvVeLxrlnpJPojWNIkQ"
TELEGRAM_USER_ID = 5155206817  # твой ID (число)

# Подключаем базу с фактами
conn = sqlite3.connect("facts.db")
cursor = conn.cursor()
cursor.execute("""
CREATE TABLE IF NOT EXISTS facts (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    statement TEXT UNIQUE,
    is_true INTEGER,
    proof TEXT
)
""")
conn.commit()

# Функция для проверки математического утверждения
def check_math(statement):
    try:
        if "==" in statement:
            left, right = statement.split("==")
            return sp.simplify(sympify(left) - sympify(right)) == 0
        else:
            return bool(sympify(statement))
    except:
        return None

# Сохраняем факт
def save_fact(statement, is_true, proof=""):
    try:
        cursor.execute("INSERT OR IGNORE INTO facts (statement, is_true, proof) VALUES (?, ?, ?)",
                       (statement, int(is_true), proof))
        conn.commit()
    except Exception as e:
        print("Ошибка при сохранении факта:", e)

# Обработка сообщений
async def handle_message(update: Update, context: ContextTypes.DEFAULT_TYPE):
    user_id = update.effective_user.id

    if user_id != TELEGRAM_USER_ID:
        await update.message.reply_text("⛔ Доступ запрещён.")
        return

    text = update.message.text.strip()
    print(f"📩 Запрос: {text}")  # Выводим запрос в Colab

    result = check_math(text)
    if result is True:
        reply = "✅ Это верно."
        save_fact(text, True, "Проверено SymPy")
    elif result is False:
        reply = "❌ Это неверно."
        save_fact(text, False, "Проверено SymPy")
    else:
        reply = "🤔 Не удалось проверить."

    print(f"💬 Ответ: {reply}")  # Выводим ответ в Colab
    await update.message.reply_text(reply)

# Запуск бота
app = ApplicationBuilder().token(TELEGRAM_TOKEN).build()
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

print("🤖 Бот Numen запущен! Жду сообщений в Telegram...")
app.run_polling()

🤖 Бот Numen запущен! Жду сообщений в Telegram...
📩 Запрос: 2+2=4
2+2=5
sqrt(16) = 4
(5*3) + (10/2) = 20
💬 Ответ: 🤔 Не удалось проверить.
